## Loading-Data

In [56]:
# Importing modules
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

In [57]:
df = pd.read_csv('AviationData.csv', encoding='ISO-8859-1')
df.head()

c:\Users\mutis\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (6,7,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.9222,-81.8781,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [58]:
# Getting more info about the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52790 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87572 non-null  object 
 14  Make                    88826 non-null

We notice some missing values on various columns

### Data-Cleaning

#### Handling missing values

Checking the percentage of missing data

In [59]:
def missing_values():
    """Function to check missing data in df
    
    Returns: (pd.Series) a series of the missing values as percentages
    """
    return df.isna().mean().sort_values(ascending=False)


missing_values()

Schedule                  0.858453
Air.carrier               0.812710
FAR.Description           0.639742
Aircraft.Category         0.636772
Longitude                 0.613304
Latitude                  0.613203
Airport.Code              0.434699
Airport.Name              0.406113
Broad.phase.of.flight     0.305606
Publication.Date          0.154924
Total.Serious.Injuries    0.140737
Total.Minor.Injuries      0.134246
Total.Fatal.Injuries      0.128261
Engine.Type               0.079616
Report.Status             0.071786
Purpose.of.flight         0.069660
Number.of.Engines         0.068445
Total.Uninjured           0.066510
Weather.Condition         0.050535
Aircraft.damage           0.035932
Registration.Number       0.014816
Injury.Severity           0.011250
Country                   0.002542
Amateur.Built             0.001147
Model                     0.001035
Make                      0.000709
Location                  0.000585
Event.Date                0.000000
Accident.Number     

Both Schedule and Air.carrier have more than 80% missing values so we can just drop the columns

In [60]:
df.drop(['Schedule', 'Air.carrier'], axis=1, inplace=True)

In [61]:
df.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Purpose.of.flight', 'Total.Fatal.Injuries', 'Total.Serious.Injuries',
       'Total.Minor.Injuries', 'Total.Uninjured', 'Weather.Condition',
       'Broad.phase.of.flight', 'Report.Status', 'Publication.Date'],
      dtype='object')

Checking the make and model which are one of our main columns

In [62]:
missing_values()[['Make', 'Model']]

Make     0.000709
Model    0.001035
dtype: float64

Only a small percentage is missing so we drop the few rows with missing entries

In [63]:
df.dropna(subset=['Make', 'Model'], inplace=True)

In [64]:
df[['Make', 'Model']].isna().sum()

Make     0
Model    0
dtype: int64

Handling missing values for the columns with continuous data by imputing the mean

In [71]:
# columns with continuous data
continuous_data_cols = [
    'Total.Fatal.Injuries',
    'Total.Serious.Injuries',
    'Total.Minor.Injuries',
    'Total.Uninjured',
    'Number.of.Engines'
]

for col in continuous_data_cols:
    mean = np.floor(df[col].mean())
    df[col].fillna(mean, inplace=True)
    
# confirm no missing values
df[continuous_data_cols].isna().sum()

Total.Fatal.Injuries      0
Total.Serious.Injuries    0
Total.Minor.Injuries      0
Total.Uninjured           0
Number.of.Engines         0
dtype: int64

Now we check other columns with categorical data

In [66]:
df['Amateur.Built'].value_counts()

No     80240
Yes     8438
Name: Amateur.Built, dtype: int64

There are only two categories in this column and No appears the most hence we impute it for the missing values

In [67]:
df['Amateur.Built'].fillna(
    df['Amateur.Built'].mode()[0],
    inplace=True
)

df['Amateur.Built'].isna().sum()

0

In [68]:
missing_values()

FAR.Description           0.640267
Aircraft.Category         0.636787
Longitude                 0.613211
Latitude                  0.613109
Airport.Code              0.434392
Airport.Name              0.405860
Broad.phase.of.flight     0.305192
Publication.Date          0.155051
Engine.Type               0.078917
Report.Status             0.071359
Purpose.of.flight         0.069140
Number.of.Engines         0.067844
Weather.Condition         0.050002
Aircraft.damage           0.035730
Registration.Number       0.014362
Injury.Severity           0.011028
Country                   0.002534
Location                  0.000586
Event.Date                0.000000
Accident.Number           0.000000
Investigation.Type        0.000000
Make                      0.000000
Model                     0.000000
Amateur.Built             0.000000
Total.Fatal.Injuries      0.000000
Total.Serious.Injuries    0.000000
Total.Minor.Injuries      0.000000
Total.Uninjured           0.000000
Event.Id            

In [69]:
df['Aircraft.Category'].value_counts()

Airplane             27580
Helicopter            3435
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 14
WSFT                     9
Powered-Lift             5
Blimp                    4
UNK                      2
ULTR                     1
Rocket                   1
Name: Aircraft.Category, dtype: int64

The Aircraft.Category is missing alot of values but it is key for our analysis so we just create a new category for the missing values

In [70]:
df['Aircraft.Category'].fillna('missing', inplace=True)
df['Aircraft.Category'].isna().sum()

0

In [73]:
df.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Purpose.of.flight', 'Total.Fatal.Injuries', 'Total.Serious.Injuries',
       'Total.Minor.Injuries', 'Total.Uninjured', 'Weather.Condition',
       'Broad.phase.of.flight', 'Report.Status', 'Publication.Date'],
      dtype='object')

Since the other columns are categorical we can fill missing values with a new category 'missing'

In [75]:
df.fillna('missing', inplace=True)
df.isna().sum().sum()

0

## Data-Aggregation

In [76]:
df[['Make', 'Model']]

,Make,Model
0,Stinson,108-3
1,Piper,PA24-180
2,Cessna,172M
3,Rockwell,112
4,Cessna,501
...,...,...
88884,PIPER,PA-28-151
88885,BELLANCA,7ECA
88886,AMERICAN CHAMPION AIRCRAFT,8GCBC
88887,CESSNA,210N


The make has some names that are in different cases

we convert make to title case and model to uppercase

In [77]:
df['Make'] = df['Make'].map(
    lambda x: x.title()
)
df['Model'] = df['Model'].map(
    lambda x: x.upper()
)
df[['Make', 'Model']]

,Make,Model
0,Stinson,108-3
1,Piper,PA24-180
2,Cessna,172M
3,Rockwell,112
4,Cessna,501
...,...,...
88884,Piper,PA-28-151
88885,Bellanca,7ECA
88886,American Champion Aircraft,8GCBC
88887,Cessna,210N


In [78]:
df['Injury.Severity'].value_counts()

Non-Fatal     67305
Fatal(1)       6159
Fatal          5248
Fatal(2)       3703
Incident       2214
              ...  
Fatal(141)        1
Fatal(54)         1
Fatal(229)        1
Fatal(270)        1
Fatal(153)        1
Name: Injury.Severity, Length: 110, dtype: int64

In [81]:
df[df['Injury.Severity'].str.contains(
    'Fatal'
)].head()[['Injury.Severity', 'Total.Fatal.Injuries']]

,Injury.Severity,Total.Fatal.Injuries
0,Fatal(2),2.0
1,Fatal(4),4.0
2,Fatal(3),3.0
3,Fatal(2),2.0
4,Fatal(1),1.0


The fatal category includes the number of fatal injuries in brackets

We can just use one category 'Fatal'

In [83]:
df['Injury.Severity'] = df['Injury.Severity'].map(
    lambda x: 'Fatal' if 'Fatal' in x else x
)
df['Injury.Severity'].value_counts()

Fatal          85098
Incident        2214
missing          979
Minor            218
Serious          173
Unavailable       95
Name: Injury.Severity, dtype: int64

Now we have one category for Fatal

In [84]:
df['Event.Date'].dtype

dtype('O')

Converting  Event date to datetime instead of object

In [90]:
df['Event.Date'] = pd.to_datetime(df['Event.Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88777 entries, 0 to 88888
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                88777 non-null  object        
 1   Investigation.Type      88777 non-null  object        
 2   Accident.Number         88777 non-null  object        
 3   Event.Date              88777 non-null  datetime64[ns]
 4   Location                88777 non-null  object        
 5   Country                 88777 non-null  object        
 6   Latitude                88777 non-null  object        
 7   Longitude               88777 non-null  object        
 8   Airport.Code            88777 non-null  object        
 9   Airport.Name            88777 non-null  object        
 10  Injury.Severity         88777 non-null  object        
 11  Aircraft.damage         88777 non-null  object        
 12  Aircraft.Category       88777 non-null  object

Creating new column for Year of occurence

In [92]:
df['year'] = df['Event.Date'].map(
    lambda x: x.year
)

In [93]:
df.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Purpose.of.flight', 'Total.Fatal.Injuries', 'Total.Serious.Injuries',
       'Total.Minor.Injuries', 'Total.Uninjured', 'Weather.Condition',
       'Broad.phase.of.flight', 'Report.Status', 'Publication.Date', 'year'],
      dtype='object')